# Prepare the data set
The goal is to prepare a folder structure like this (where which img is a 2D image)




```
data_yapic/
        └───input/
        │       img1.tiff
        │       img2.tiff
        │       img3.tiff
        │       ...
        │   
        └───target/
        │       img1_mask.tiff
        │       img2_mask.tiff
        │       img3_mask.tiff
        │       ...
```

### From partial annotated tiff stack
For example using the `visualization_napari_workflow.ipynb`

In [7]:
import os
import numpy as np

from pathlib import Path
from tifffile import imread, imwrite

In [3]:
# load raw image
path_im = Path('../data/louise/chip-2.tif')
im = imread(path_im)

# load label image
path_label = Path('../data/louise/chip-2-labels.tif')
labels = imread(path_label)

In [3]:
# load raw image
path_im = Path('../data/placozoan-movie.tif')
im = imread(path_im)

# load label image
path_label = Path('../data/results/placozoan-movie_masked.tif')
labels = imread(path_label)

In [5]:
# create empty folder if it doesn't exist to save the data
dataset_path = Path('../data/data_louise_yapic')
if not os.path.isdir(dataset_path):
    os.mkdir(dataset_path)
    os.mkdir(dataset_path/"input")
    os.mkdir(dataset_path/"target")

In [8]:
# populate the folders

for i, t in enumerate(labels):
    if 1 in t: # if this slide has labels
        # t[t!=1]=2
        imwrite(dataset_path/"input"/(str(i)+".tif"), im[i].astype(np.uint8), dtype=np.uint8)
        imwrite(dataset_path/"target"/(str(i)+".tif"), t.astype(np.uint8), dtype=np.uint8)

In [31]:
# check results
import napari
viewer = napari.Viewer()

# Train YAPIC Model

In [23]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2022-07-04 23:42:30.460539: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-04 23:42:30.463451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:42:30.463461: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[]


2022-07-04 23:42:31.506126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 23:42:31.506606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:42:31.506664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-04 23:42:31.506714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-04 23:42:31.508656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2022-07-04 23:41:20.181031: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-04 23:41:20.183881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:41:20.183890: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17863907655473116994
xla_global_id: -1
]


2022-07-04 23:41:21.227824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-04 23:41:21.281605: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 23:41:21.282024: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:41:21.282085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared o

In [17]:
from yapic.session import Session

img_path = str(dataset_path/"input")+'/*.tif'
label_path = str(dataset_path/"target")+'/*.tif'

model_size_xy = (1, 572, 572)

t = Session()
t.load_training_data(img_path, label_path)
t.make_model('unet_2d', model_size_xy)

t.define_validation_data(0.2) # 80% training data, 20% validation data

t.train(max_epochs=10,
        steps_per_epoch=48,
        log_filename='../models/log.csv')

t.model.save('../models/my_model.h5')




Import taining dataset:
TiffConnector object
image filepath: /home/clement/Documents/centuri-hackaton/placozoa-tracking/data/data_yapic/input
label filepath: /home/clement/Documents/centuri-hackaton/placozoa-tracking/data/data_yapic/target
nr of images: 5
labelvalue_mapping: [{1: 1, 2: 2}]


Initialize model unet_2d


2022-07-04 23:11:28.735696: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 23:11:28.736309: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:11:28.736397: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-04 23:11:28.736458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-04 23:11:28.763706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Compute tile positions: [########################################] 5/5
Compute tile positions: [########################################] 5/5

Compute tile positions: [########################################] 5/5). This takes a few minutes...Compute tile positions: [........................................] 0/5
Compute tile positions: [########################################] 5/5


/home/clement/miniconda3/envs/placozoa/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/home/clement/miniconda3/envs/placozoa/lib/python3.8/site-packages/yapic/session.py:258: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.history = self.model.fit_generator(


Epoch 1/10
48/48 [==============================] - ETA: 0s - loss: 0.6267 - accuracy: 0.7931

KeyboardInterrupt: 